In [ ]:
import warnings

warnings.filterwarnings("ignore")
from pathlib import Path
import pandas as pd
from tbdynamics.calibration.utils import (
    get_bcm,
    calculate_covid_diff_quantiles,
    plot_covid_scenarios_comparison,
    plot_output_ranges,
)
import arviz as az
from tbdynamics.inputs import load_targets
from tbdynamics.constants import params_name, quantiles
from estival.sampling import tools as esamp
import numpy as np

In [ ]:
OUT_PATH = Path.cwd() / 'runs/r2508'
idata = az.from_netcdf(OUT_PATH / 'calib_full_out.nc')

In [ ]:
params = {
    "start_population_size": 2000000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
}
covid_effects = {
    'detection_reduction':True,
    'contact_reduction':True
}

In [ ]:
# spaghetti = pd.read_hdf(OUT_PATH / 'results.hdf', 'spaghetti')
# quantile_outputs = esamp.quantiles_for_results(spaghetti, quantiles)
targets = load_targets()
burnt_idata = idata.sel(draw=np.s_[50000:])
idata_extract = az.extract(burnt_idata, num_samples=500)

In [ ]:
diff_outputs = calculate_covid_diff_quantiles(params, idata_extract, ['incidence', 'mortality_raw'],[2021.0,2022.0],1)

In [ ]:
plot_covid_scenarios_comparison(diff_outputs, indicators=["incidence", "mortality_raw"], years=[2021.0, 2022.0], plot_type="abs", n_cols=1)

In [ ]:
spaghetti_res = esamp.model_results_for_samples(idata_extract, get_bcm(params, covid_effects))

In [ ]:
quantile_outputs = esamp.quantiles_for_results(spaghetti_res.results, quantiles)

In [ ]:
plot_output_ranges(quantile_outputs,targets, ['incidence','mortality_raw'], 1, 2010, 2025)

In [ ]:
covid_effects = {
    'detection_reduction':False,
    'contact_reduction':False
}

In [ ]:
spaghetti_res = esamp.model_results_for_samples(idata_extract, get_bcm(params, covid_effects))
quantile_outputs = esamp.quantiles_for_results(spaghetti_res.results, quantiles)
plot_output_ranges(quantile_outputs,targets, ['incidence','mortality_raw'], 1, 2010, 2025)